# Clustering Businesses Descriptions of their Principal Activities

The goal is to create a robust clustering pipeline - it should include creating an "optimal" clustering, validating and filtering the resulting clusters to those that contain real data, and then summarising the resulting text in some human-readable way that allows the researcher to create a definition for a cluster without having to go back and read the original documents.

In [1]:
import gensim
import re

import numpy as np
import pandas as pd

import gensim.models.ldamulticore as LDA

from gensim.utils import simple_preprocess
from nltk.stem import SnowballStemmer
from gensim.parsing.preprocessing import STOPWORDS

from nltk.stem.porter import *

# Define which stemmer to use in the pipeline later
stemmer = PorterStemmer()

In [4]:
pa_df = pd.read_csv("./data/companieshousedescriptions.csv").drop("Unnamed: 0", axis=1)

## 1.  Data exploration/explanation

This data contains descriptions of company principal activities submitted to Companies House within balance sheets, for September 2017 - August 2018.  The descriptions are usually very brief, just a sentence or so.  They're also usually very vague (eg; "consulting" or "financial services").  There's over 700,000 documents but far fewer actually contain information!

In [5]:
pa_df.head()

balancesheetdate companieshouseregisterednumber  \
0       2017-06-30                       08028272   
1       2017-06-30                       04761288   
2       2017-08-31                       10314494   
3       2016-12-31                       03127240   
4       2017-06-30                       05735108   

                      descriptionprincipalactivities  
0  The principal activity of the company is Motor...  
1               No description of principal activity  
2               No description of principal activity  
3               No description of principal activity  
4               No description of principal activity

In [6]:
 len(pa_df)

747216

In [7]:
pa_cleaned_df = pa_df[pa_df['descriptionprincipalactivities'] != "No description of principal activity"]
pa_cleaned_df.head()

balancesheetdate companieshouseregisterednumber  \
0        2017-06-30                       08028272   
6        2017-06-30                       05832111   
8        2017-08-31                       02962718   
26       2017-07-31                       06658591   
28       2017-07-31                       07326706   

                       descriptionprincipalactivities  
0   The principal activity of the company is Motor...  
6   The principal activity of the company is Desig...  
8                                 software production  
26  Other letting and operating of leased real estate  
28  The principal activity of the company is Sale ...

In [8]:
len(pa_cleaned_df)

219386

After cleaning out the obvious null value we end up with over 200,000 records, which isn't so bad - there's likely to be other effectively null values (in fact, I KNOW there is from other work), but rather than try to identify them individually we'll see if we can clean them out in an automated manner as part of the clustering process.

In [9]:
pa_cleaned_df['numcharacters'] = pa_cleaned_df['descriptionprincipalactivities'].apply(lambda x: len(str(x)))

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
pa_cleaned_df['numwords'] = pa_cleaned_df['descriptionprincipalactivities'].apply(lambda x: len(str(x).split()))

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
pa_cleaned_df.head()

balancesheetdate companieshouseregisterednumber  \
0        2017-06-30                       08028272   
6        2017-06-30                       05832111   
8        2017-08-31                       02962718   
26       2017-07-31                       06658591   
28       2017-07-31                       07326706   

                       descriptionprincipalactivities  numcharacters  numwords  
0   The principal activity of the company is Motor...             65        10  
6   The principal activity of the company is Desig...             59         9  
8                                 software production             19         2  
26  Other letting and operating of leased real estate             49         8  
28  The principal activity of the company is Sale ...             65        11

## 2.  Preprocess all of the text descriptions

This will be fed into the TF-IDF text representation.  Since it's business descriptions we only really need the keywords cluster them by, all of the stopwords are irrelevant and simply inflate the vocabulary, and semantic meanings/conflations are largely irrelevant.  So, we're going to strip stopwords and stem the vocabulary before clustering.  Lemmatisation is an option but it's hard to know if it would obfusticate industry-specific technical terms.

There's a good guide on this process at https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24, though I'm going to be tailoring it a bit for myself.

NB;  On the puny laptop I use while travelling (1.2Ghz, 4GB ram) even just stemming can take quite a while!

In [12]:
# A quick utility function to pre-process the text
def preprocess_desc(description):
    return( [stemmer.stem(token) for token in simple_preprocess(str(description)) if token not in STOPWORDS] )

In [13]:
pa_cleaned_df['tokens'] = pa_cleaned_df['descriptionprincipalactivities'].apply(preprocess_desc)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
pa_cleaned_df['numtokens'] = pa_cleaned_df['tokens'].apply(len)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
pa_cleaned_df.head()

balancesheetdate companieshouseregisterednumber  \
0        2017-06-30                       08028272   
6        2017-06-30                       05832111   
8        2017-08-31                       02962718   
26       2017-07-31                       06658591   
28       2017-07-31                       07326706   

                       descriptionprincipalactivities  numcharacters  \
0   The principal activity of the company is Motor...             65   
6   The principal activity of the company is Desig...             59   
8                                 software production             19   
26  Other letting and operating of leased real estate             49   
28  The principal activity of the company is Sale ...             65   

    numwords                                            tokens  numtokens  
0         10      [princip, activ, compani, motorsport, engin]          5  
6          9          [princip, activ, compani, design, engin]          5  
8          2                                [softwar, product]          2  
26         8                    [let, oper, leas, real, estat]          5  
28        11  [princip, activ, compani, sale, offic, furnitur]          6

In [16]:
len(pa_cleaned_df)

219386

In [17]:
pa_cleaned_df.to_csv("./data/companieshousedescriptions_cleaned.csv")

## 3.  Create TF-IDF text representation

In [18]:
# Create the vocabulary record
dictionary = gensim.corpora.Dictionary(pa_cleaned_df['tokens'])

In [19]:
# Remove extreme values (words that are too rare, too common)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

In [20]:
# Create a BOW model
bow_corpus = [dictionary.doc2bow(doc) for doc in pa_cleaned_df['tokens']]

In [21]:
# From that create the TF-IDF model
tfidf = gensim.models.TfidfModel(bow_corpus)
pa_cleaned_df['corpus_tfidf'] = tfidf[bow_corpus]

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
pa_cleaned_df['corpus_tfidf'].head()

0     [(0, 0.3639053457906437), (1, 0.93143593408510...
6     [(0, 0.690464842484867), (2, 0.7233659525387879)]
8     [(3, 0.6762904085987101), (4, 0.73663510861035...
26    [(5, 0.43307864762118525), (6, 0.5161195419947...
28    [(10, 0.6891561579742794), (11, 0.605489898421...
Name: corpus_tfidf, dtype: object

## 4.  Fit LDA to TF-IDF model

This should perform better than using mere BOW, because the scores/values are already emphasised if they are discriminative.  This should reduce noise (meaning here words that appear in many different topics).

Ironically, I need to be careful I don't filter out data/words I'm trying to filter later using the clustering...

In [25]:
# Fit the lda model, with 10 topics
lda_model_tfidf = gensim.models.LdaMulticore(pa_cleaned_df['corpus_tfidf'],
                                             num_topics=10,
                                             id2word=dictionary,
                                             passes=2,
                                             workers=2)

In [26]:
# Let's take a quick look at the topics picked out
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.053*"softwar" + 0.037*"repair" + 0.030*"vehicl" + 0.030*"motor" + 0.026*"develop" + 0.025*"medic" + 0.024*"sell" + 0.024*"public" + 0.024*"stuff" + 0.023*"restaur"
Topic: 1 Word: 0.200*"dormant" + 0.136*"trade" + 0.074*"year" + 0.057*"contractor" + 0.036*"clean" + 0.020*"end" + 0.018*"subsequ" + 0.017*"electr" + 0.016*"period" + 0.014*"non"
Topic: 2 Word: 0.245*"consult" + 0.097*"servic" + 0.060*"contract" + 0.059*"provis" + 0.058*"financi" + 0.025*"health" + 0.019*"secur" + 0.016*"provid" + 0.016*"joineri" + 0.016*"veterinari"
Topic: 3 Word: 0.209*"descript" + 0.208*"disclos" + 0.067*"busi" + 0.036*"support" + 0.025*"work" + 0.023*"classifi" + 0.022*"plumb" + 0.022*"servic" + 0.021*"educ" + 0.020*"domest"
Topic: 4 Word: 0.189*"manag" + 0.069*"construct" + 0.067*"consult" + 0.049*"properti" + 0.033*"gener" + 0.026*"nan" + 0.025*"mainten" + 0.023*"recruit" + 0.022*"build" + 0.019*"hairdress"
Topic: 5 Word: 0.112*"properti" + 0.061*"develop" + 0.060*"principl" + 0.056*"i

# Conclusions

Even without plotting it, we can see some keywords are ending up grouped oddly.  Plumbing, motor, electric and other maintenance/mechanical type jobs end up in different groups.  I'd suggest that a word2vec powered model would give use a much better clustering of these topics, because the meaning and context of words provides additional information with these extremely short sentences.